In [71]:
# Para llevar un registro de los cambios
%pip install -q comet_ml
import comet_ml
comet_ml.init(project_name='Tarea 03')

In [72]:
# Librerías
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras import regularizers
from keras.callbacks import ModelCheckpoint

In [73]:
experiment = comet_ml.Experiment(
    auto_histogram_weight_logging=True,
    auto_histogram_gradient_logging=True,
    auto_histogram_activation_logging=True,
    log_code=True,
)
experiment.set_name("Regularización_L1L2")

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/gurumasterahb/tarea-03/d0c4c43efebc43f9afe8c5fb0ac65275
COMET INFO:   Others:
COMET INFO:     Name : Regularización_L1L2
COMET INFO:   Parameters:
COMET INFO:     batch_size    : 10
COMET INFO:     epochs        : 20
COMET INFO:     learning_rate : 0.01
COMET INFO:     loss          : categorical_crossentropy
COMET INFO:     optimizer     : adam
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     notebook            : 2
COMET INFO:     os packages         : 1
COMET INFO:     source_code         : 1
COMET INFO: 
COMET WARNIN

In [74]:
# Carga el dataset de MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [75]:
# Convierte y normaliza los datos de entrada
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [76]:
# to_categorical convierte las etiquetas (train, test) en matrices one-hot
num_classes = 10
y_trainc = keras.utils.to_categorical(y_train, num_classes)
y_testc = keras.utils.to_categorical(y_test, num_classes)

In [77]:
  # Diccionario con los parámetros que se registrarán en comet
  parameters = {
      'epochs' : 20,
      'batch_size' : 10,
      'learning_rate' : 0.010,
      'loss' : 'categorical_crossentropy',
      'optimizer' : 'adam'
  }
  experiment.log_parameters(parameters)

In [78]:
# Creación del modelo
model = Sequential()
# Es como una "primera capa" de tres capas
model.add(Input(shape = (28,28))) # Matrices de entrada de 28x28 (imáegenes)
model.add(Flatten()) # Convierte las matrices de 28x28 en matrices de 784x1
model.add(Dense(784, activation = 'sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(30, activation = 'relu', kernel_regularizer = regularizers.L1L2(0.01, 0.01))) # Segunda capa
model.add(Dense(10, activation = 'softmax')) # Tercera capa

model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_7 (Flatten)         (None, 784)               0         
                                                                 
 dense_17 (Dense)            (None, 784)               615440    
                                                                 
 dropout_7 (Dropout)         (None, 784)               0         
                                                                 
 dense_18 (Dense)            (None, 30)                23550     
                                                                 
 dense_19 (Dense)            (None, 10)                310       
                                                                 
Total params: 639300 (2.44 MB)
Trainable params: 639300 (2.44 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [79]:
# Checkpoint que guarda el mejor modelo (cuando la función de costo alcanza un mínimo)
checkpoint = ModelCheckpoint(filepath = 'best_model.hdf5', monitor = 'val_loss', verbose = 1, save_best_only = True, mode = 'min')

In [80]:
# optimizer = RMSprop(learning_rate = 0.0005)
optimizer = 'adam'
model.compile(loss = parameters['loss'], optimizer = optimizer, metrics = ['accuracy'])
model.fit(x_train, y_trainc, batch_size = parameters['batch_size'], epochs = parameters['epochs'], verbose = 1, validation_data = (x_test, y_testc), callbacks = [checkpoint])

Epoch 1/20
5998/6000 [============================>.] - ETA: 0s - loss: 1.1929 - accuracy: 0.8372
Epoch 1: val_loss improved from inf to 0.77120, saving model to best_model.hdf5
6000/6000 [==============================] - 71s 12ms/step - loss: 1.1928 - accuracy: 0.8373 - val_loss: 0.7712 - val_accuracy: 0.9101
Epoch 2/20


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


5997/6000 [============================>.] - ETA: 0s - loss: 0.7288 - accuracy: 0.9060
Epoch 2: val_loss improved from 0.77120 to 0.62476, saving model to best_model.hdf5
6000/6000 [==============================] - 65s 11ms/step - loss: 0.7287 - accuracy: 0.9060 - val_loss: 0.6248 - val_accuracy: 0.9236
Epoch 3/20
5997/6000 [============================>.] - ETA: 0s - loss: 0.6518 - accuracy: 0.9193
Epoch 3: val_loss improved from 0.62476 to 0.62277, saving model to best_model.hdf5
6000/6000 [==============================] - 64s 11ms/step - loss: 0.6518 - accuracy: 0.9193 - val_loss: 0.6228 - val_accuracy: 0.9257
Epoch 4/20
5999/6000 [============================>.] - ETA: 0s - loss: 0.6078 - accuracy: 0.9286
Epoch 4: val_loss improved from 0.62277 to 0.53375, saving model to best_model.hdf5
6000/6000 [==============================] - 61s 10ms/step - loss: 0.6079 - accuracy: 0.9286 - val_loss: 0.5338 - val_accuracy: 0.9400
Epoch 5/20
5997/6000 [============================>.] - ETA:

In [81]:
experiment.log_model('MNIST1', 'best_model.hdf5')
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/gurumasterahb/tarea-03/b1a1794e2ddf457e81bec41a7b7e027c
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [20]                  : (0.8372666835784912, 0.9592666625976562)
COMET INFO:     batch_accuracy [12000]         : (0.10000000149011612, 1.0)
COMET INFO:     batch_loss [12000]             : (0.29590702056884766, 13.603830337524414)
COMET INFO:     epoch_duration [20]            : (60.42000262599868, 71.8793612010013)
COMET INFO:     loss [20]                      : (0.4467546343803406, 1.1927858591079712)
COMET INFO:     val_accuracy [20]              : (0.910099983215332, 0.9649999737739563)
COMET INFO:    